In [2]:
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import ray
from random import sample

In [55]:
def netExp(R,P,x,b):
    k = np.sum(x);
    k0 = 0;
    y = [];
    
    while k > k0:
        k0 = np.sum(x);
        y = (np.dot(R.transpose(),x) == b);
        y = y.astype('int');
        x_n = np.dot(P,y) + x;
        x_n = x_n.astype('bool');
        x = x_n.astype('int');
        k = np.sum(x);
    return x,y

def netExp_trace(R,P,x,b):
    
    X = []
    Y = []
    
    X.append(x)
    k = np.sum(x);
    k0 = 0;
    y = [];
    
    Y.append(y)
    
    while k > k0:
        k0 = np.sum(x);
        y = (np.dot(R.transpose(),x) == b);
        y = y.astype('int');
        x_n = np.dot(P,y) + x;
        x_n = x_n.astype('bool');
        x = x_n.astype('int');
        k = np.sum(x);
        X.append(x)
        Y.append(y) 
    return X,Y


def parse_reaction_trace(reaction_trace,network):
    rxns_list = []
    for i in range(1,len(reaction_trace)):
        idx = reaction_trace[i].nonzero()[0]
        rxns = list(network.iloc[:,idx])
        rxns = pd.DataFrame(rxns,columns = ['rn','direction'])
        rxns['iter'] = i
        rxns_list.append(rxns)    
    rxns_list = pd.concat(rxns_list,axis=0)
    return rxns_list

class GlobalMetabolicNetwork:
    
    def __init__(self):
        # load the data
        network = pd.read_csv('../networkExpansionPy/assets/KEGG/network_full.csv')
        cpds = pd.read_csv('../networkExpansionPy/assets/compounds/cpds.tab',sep='\t')
        thermo = pd.read_csv('../networkExpansionPy/assets/reaction_free_energy/kegg_reactions_CC_ph7.0.csv',sep=',')
        self.network = network
        self.compounds = cpds
        self.thermo = thermo
        self.temperature = 50
        self.seedSet = None;
        
    def set_ph(self,pH):
        if ~(type(pH) == str):
            pH = str(pH)
        try:
            thermo = pd.read_csv('../networkExpansionPy/assets/reaction_free_energy/kegg_reactions_CC_ph' + pH + '.csv',sep=',')
            self.thermo = thermo
        except Exception as error:
            print('Failed to open pH files (please use 5.0-9.0 in 0.5 increments)')    
    
    
    def pruneInconsistentReactions(self):
        # remove reactions with qualitatively different sets of elements in reactions and products
        consistent = pd.read_csv('../networkExpansionPy/assets/reaction_sets/reactions_consistent.csv')
        self.network = self.network[self.network.rn.isin(consistent.rn.tolist())]
        
    def pruneUnbalancedReactions(self):
        # only keep reactions that are elementally balanced
        balanced = pd.read_csv('../networkExpansionPy/assets/reaction_sets/reactions_balanced.csv')
        self.network = self.network[self.network.rn.isin(balanced.rn.tolist())]
        
    
    def convertToIrreversible(self):
        network = self.network
        rn = network[['rn']]
        cid = network[['cid']]
        s = network[['s']]
        rn_f = rn + ''
        rn_b = rn + ''
        rn_f['direction'] = 'forward'
        rn_b['direction'] = 'reverse'
        
        nf = cid.join(rn_f).join(s)
        nb = cid.join(rn_b).join(-s)
        self.network = pd.concat([nf,nb],axis=0) 
    
    def setMetaboliteBounds(self,ub = 1e-1,lb = 1e-6): 
        
        self.network['ub'] = ub
        self.network['lb'] = lb
      
    
    def pruneThermodynamicallyInfeasibleReactions(self,keepnan = False):
        fixed_mets = ['C00001','C00080']

        RT = 0.008309424 * (273.15+self.temperature)
        rns  = []
        dirs = []
        dgs = []
        for (rn,direction), dff in self.network.groupby(['rn','direction']):
            effective_deltaG = np.nan
            if rn in self.thermo['!MiriamID::urn:miriam:kegg.reaction'].tolist():
                deltaG = self.thermo[G.thermo['!MiriamID::urn:miriam:kegg.reaction'] == rn]['!dG0_prime (kJ/mol)'].values[0]
                if direction == 'reverse':
                    deltaG = -1*deltaG

                dff = dff[~dff['cid'].isin(fixed_mets)]
                subs = dff[dff['s'] < 0]
                prods = dff[dff['s'] > 0];
                k = np.dot(subs['ub'].apply(np.log),subs['s']) + np.dot(prods['lb'].apply(np.log),prods['s'])

                effective_deltaG = RT*k + deltaG

            dgs.append(effective_deltaG)
            dirs.append(direction)
            rns.append(rn)

        res = pd.DataFrame({'rn':rns,'direction':dirs,'effDeltaG':dgs})
        if ~keepnan:
            res = res.dropna()
        
        res = res[res['effDeltaG'] < 0].set_index(['rn','direction'])
        res = res.drop('effDeltaG',axis=1)
        self.network = res.join(self.network.set_index(['rn','direction'])).reset_index()
    
    def initialize_metabolite_vector(self,seedSet):
        if self.seedSet is None:
            print('No seed set')
        else:
            network = self.network.pivot_table(index='cid',columns = ['rn','direction'],values='s').fillna(0)
            x0 = np.array([x in seedSet for x in network.index.get_level_values(0)]) * 1;        
            return x0
        
    

In [123]:
# build global metabolic network
G = GlobalMetabolicNetwork()
G.pruneInconsistentReactions()
G.convertToIrreversible()

#G.pruneUnbalancedReactions()
#G.set_ph(7.0)
#G.convertToIrreversible()
#G.setMetaboliteBounds(ub=1e-1,lb=1e-6)
#G.pruneThermodynamicallyInfeasibleReactions(keepnan=False)

In [124]:
# construct params for network expansion
network = G.network.pivot_table(index='cid',columns = ['rn','direction'],values='s').fillna(0)
S = network.values
R = (S < 0)*1
P = (S > 0)*1
b = sum(R)

# sparsefy data
R = csr_matrix(R)
P = csr_matrix(P)
b = csr_matrix(b)
b = b.transpose()

In [150]:

size_of_random_sample = 6
fixed_metabolites = ['C00001','C00011','C00080','C00014','C00009','C00283']
mets_population = [x for x in network.index.get_level_values(0).tolist() if x not in fixed_metabolites]

numSims = 1000;
sims = []
for i in range(numSims):
    
    seedSet = sample(mets_population,size_of_random_sample) + fixed_metabolites
    x0 = np.array([x in seedSet for x in network.index.get_level_values(0)]) * 1
    x0 = csr_matrix(x0)
    x0 = x0.transpose()
    met_trace,reaction_trace = netExp_trace(R,P,x0,b)
    rxn_iter = parse_reaction_trace(reaction_trace,network)
    sim = rxn_iter.groupby('rn').min()['iter']
    sim = pd.DataFrame(sim)
    sim.columns = [i]
    sims.append(sim)
    
sims = pd.concat(sims,axis=1)
sims.to_csv('randnetworkExpansion.ConsistentNetework.NoThermo.CHOSNPfixedMets.1000Samples.11202020.csv')

/Users/joshuagoldford/miniconda2/envs/informatics/lib/python3.6/site-packages/scipy/sparse/compressed.py:226: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


KeyboardInterrupt: 

In [151]:
sims = pd.concat(sims,axis=1)

/Users/joshuagoldford/miniconda2/envs/informatics/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [152]:
sims

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
R00004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
R00005,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
R00006,2.0,4.0,2.0,10.0,8.0,3.0,5.0,3.0,2.0,3.0,...,6.0,9.0,2.0,10.0,8.0,7.0,5.0,10.0,5.0,8.0
R00008,2.0,4.0,2.0,10.0,8.0,3.0,5.0,3.0,2.0,3.0,...,6.0,9.0,2.0,10.0,8.0,7.0,5.0,10.0,5.0,8.0
R00009,3.0,3.0,3.0,4.0,2.0,4.0,2.0,4.0,4.0,2.0,...,4.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,3.0,4.0
R00010,8.0,6.0,7.0,16.0,14.0,9.0,5.0,8.0,9.0,10.0,...,2.0,15.0,9.0,16.0,14.0,11.0,11.0,16.0,6.0,15.0
R00012,13.0,3.0,14.0,18.0,16.0,10.0,14.0,12.0,10.0,10.0,...,16.0,17.0,14.0,18.0,16.0,16.0,10.0,18.0,3.0,17.0
R00013,4.0,4.0,5.0,6.0,4.0,6.0,4.0,2.0,5.0,2.0,...,6.0,5.0,4.0,6.0,4.0,6.0,5.0,6.0,5.0,5.0
R00014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R00015,9.0,7.0,8.0,17.0,15.0,10.0,6.0,9.0,10.0,11.0,...,3.0,16.0,10.0,17.0,15.0,12.0,12.0,17.0,7.0,16.0
